单例

一个类产生一个实例 保证程序线程安全 多条线程并行执行程序 共享数据
保证线程安全方法： 给共享资源加锁。保证资源只被一个线程占用

特点： 
构造函数和析构函数是private 禁止外部构造和析构
拷贝构造函数和赋值构造函数是private，禁止外部拷贝和赋值，保证实例的唯一性
类中有一个获取实例的静态方法getInstance()，可以全局访问

g++  -std=c++11 -o singleton singleton.cpp

        产生undefined reference to `pthread_detach' 或
        为定义的引用pthread_create时
        编译加上-pthread

单例懒汉模式结果：
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
threadID = 3
构造函数
实例内存地址为 ： 构造函数
实例内存地址为 ： 0x7f78d80008c00x7f78e00008c0
threadID = 0x2
实例内存地址为 ： 0x7f78d80008c0
构造函数
实例内存地址为 ： 0x7f78dc0008c0

......
析构函数
main() finished
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

可知：单例构造函数创建了两个实例，内存地址分别是0x7f78d80008c0 和 0x7f78dc0008c0
因此普通懒汉模式 只适合单进程不适合多进程
是线程不安全的


加锁的懒汉单例模式singleton_2.cpp：
运行结果:
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
...main()...
...create..threads[0]
...create..threads[1]
...create..threads[2]
threadid = [...create..threads[0]3
构造函数
Address = 0x7fa6200008c0
threadid = [2]
Address = 0x7fa6200008c0
]
...create..threads[threadid = [4]
threadid = [3]
Address = 0x7fa6200008c0
1]
Address = 0x7fa6200008c0
析构函数
main() finished
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
可知：只创建了一个实例，内存地址为0x7fa6200008c0 是线程安全的

获取当前时间
#include <ctime>
auto currentTime = std::time(NULL);

struct tm local_time;
localtime_r(&currentTime,&local_time);

uint32_t hour = (uint32_t)local_time.tm_hour;
uint32_t min  = (uint32_t)local_time.tm_min;
uint32_t sec  = (uint32_t)local_time.tm_sec;

return (hour * 10000) + (min * 100) + sec ;

饿汉单例模式（本身是线程安全的）：

初始化静态指针变量就构造初始化函数
Singleton* Singleton::m_SingleInstance = new(nothrow) Singleton;



singleton_safe.cpp运行结果：
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
..main()..
create_thread[0]
create_thread[1]
thread  id = [0]
thread addresscreate_thread[2]
0x664010
create_thread[3]
create_thread[4]
thread  id = [1]
thread address0x664010
thread  id = [2]
thread address0x664010
thread  id = [3]
thread address0x664010
..all_finished..
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
只创建了一个实例，是线程安全的

懒汉式是以时间换空间，适应于访问量较小时；推荐使用内部静态变量的懒汉单例，代码量少
饿汉式是以空间换时间，适应于访问量较大时，或者线程比较多的的情况

auto 和 decltype

auto的使用必须马上初始化，否则无法推导出类型
auto在一行定义多个变量时，各个变量的推导不能产生二义性，否则编译失败
auto不能用作函数参数
在类中auto不能用作非静态成员变量
auto不能定义数组，可以定义指针
auto无法推导出模板参数


auto func = [&] {
    cout << "xxx";
};
auto asyncfunc = std::async(std::launch::async, func);

std::async 异步接口

获取线程函数的返回结果时，一般需要等到thread.join()之后，读取线程函数中的变量
异步接口std::async 可以获取线程函数的执行结果

std::async会自动创建一个线程去调用线程函数
返回一个std::future
这个future中存储了线程函数返回的结果，当需要这个线程函数时，直接从future中获取

std::async解耦了线程的创建和执行，可以在需要的时候获取异步操作的结果
其次还提供了线程的创建策略，可以通过延迟加载的方式创建线程

std::future 提供了访问异步操作结果的机制
因为一个异步操作不能马上获取操作结果，只能在未来某个时刻获取
可以通过同步等待的方式来获取结果 通过查询future的状态来获取异步操作的结果
future_status的三种状态：
    deferred 异步操作还没开始
    ready 异步操作已经完成
    timeout 异步操作超时


//查询future的状态
std::future_status status;
    do {
        status = future.wait_for(std::chrono::seconds(1));
        if (status == std::future_status::deferred) {
            std::cout << "deferred\n";
        } else if (status == std::future_status::timeout) {
            std::cout << "timeout\n";
        } else if (status == std::future_status::ready) {
            std::cout << "ready!\n";
        }
    } while (status != std::future_status::ready);
获取future结果有三种方式：get、wait、wait_for，其中get等待异步操作结束并返回结果，wait只是等待异步操作完成，没有返回值，wait_for是超时等待返回结果

std::async的基本用法
std::future<int> f1 = std::async(std::launch::async,[](){
    return 8;
});

cout << f1.get() << endl;
-------------------------------------------------------------
std::future<int> f2 = std::async(std::launch::async,[](){
    cout << 8 << endl; 
});
f2.wait();
//数据类型可以使用auto

auto用于推导变量类型，而decltype则用于推导表达式类型


int func() { return 0; }
decltype(func()) i; // i为int类型

int x = 0;
decltype(x) y; // y是int类型
decltype(x + y) z; // z是int类型

//decltype不会像auto一样忽略引用和cv属性，decltype会保留表达式的引用和cv属性

cont int &i = 1;
int a = 2;
decltype(i) b = 2; // b是const int&

exp是表达式，decltype(exp)和exp类型相同
exp是函数调用，decltype(exp)和函数返回值类型相同
其它情况，若exp是左值，decltype(exp)是exp类型的左值引用


int a = 0, b = 0;
decltype(a + b) c = 0; // c是int，因为(a+b)返回一个右值
decltype(a += b) d = c;// d是int&，因为(a+=b)返回一个左值

d = 20;
cout << "c " << c << endl; // 输出c 20//因为 int&d = c

auto 和decltype的配合使用 一般用于推导函数返回值类型
template<typename T,typename U>
auto add(T t,U u) -> decltype(t + u){
    return t + u;
}
返回值后置类型语法：解决函数返回值类型依赖于参数 但难以确定返回值类型的问题

----------cpp沉思录-----------

类和继承 oop
指使用继承和动态绑定的编程方式

继承是一种抽象，允许在某些时候忽略相似对象间的差异，又在其他时候利用这些差异

cpp在编译时标明哪些类型时相似的
因此在编译那些将来可能动态绑定的函数调用 时检查类型
采用这种编译时检查的方式，时因为cpp可以为动态绑定的函数快速调用生成代码

只有在程序通过指向基类对象的指针或者 基类对象的引用调用虚函数时
才会发生运行时的多态现象

容器——无论时类似于数组或者结构体的内建容器，还是用户自定义容器类（只能获得编译时类型已知的元素值）
如果有一系列类之间存在继承关系，当我们需要创建 复制和存储对象
而这些对象的确切类型只有到运行时才能知道，则这种编译时的检查会带来一些麻烦
解决方法： 增加一个间接层
通常C模型会建议采用指针来实现这个间接层
这样做会给这些类的用户带来负面影响，使得它们必须参与内存管理

cpp使用定义一个类来提供并隐藏这个间接层
这个类通常叫做句柄类handle
句柄类采用：
    将一个单一类型的对象与 一个与之有某种特定继承关系的 任意类型的对象捆绑起来

    句柄可以让我们忽略正在处理的对象的准确类型，同时避免指针带来的关于内存管理的麻烦

句柄的一个常见用途是： 通过避免不必要的复制来优化内存管理

类的构造函数：
    有些类太简单，它们的结构就是它们的接口
    所以不需要构造函数
    有些复杂的类，就需要构造函数来隐藏它们的内部工作方式

数据成员私有：
    公有的就无法控制何时访问这些成员
    使用函数而不是变量，在还允许读取访问的时候 能够很容易地阻止写入访问

类的无参构造函数：
    如果类已经有了构造函数，此时想声明该类的对象时 可以不必显示地初始化它们
    则必须显示地写一个无参的构造函数
    ps：如果一个类需要一个显示构造函数，则生成该类对象的数组时非法的如Point pa[100]
    如果想把类的所有实例都初始化。该考虑付出的代价，是否值得为此禁止对象数组

构造函数初始化所有的数据成员：
    构造函数的用途：用一种明确定义的状态来设置对象
    对象的状态由对象的数据成员进行反应
    因此，每个构造函数都要负责为所有的数据成员设置经过明确定义的值
    but：类有一些数据成员，只在它们的对象存在了一定时间后才有意义

类的析构函数：
    不是所有的构造函数的类都需要析构函数
    例如：表示复数的类即使有构造函数也可能不需要析构函数
    看该类是否分配了资源，这些资源会不会由成员函数自动释放
    尤其是：一些构造函数中包含了new表达式的类 通常需要在析构函数中加上相应的delete表达式

类的虚析构函数：
    有些类需要虚析构函数是为了声明它们的析构函数是虚的
    派生类不需要虚析构函数
    任何虚函数只在继承的情况下才有用
    eg：class B{} class D:public B{}
    此时若有向D类型对象的B* 指针执行delete表达式 
    B* bp=new D;
    delete bp;
    就需要给B加上一个虚析构函数
    即使B和D都没有虚函数 也需要加上

类的复制构造函数：
    关键在于复制该类的对象 是否相当于复制 其数据成员和基类对象
    如果并不相当 就需要复制构造函数
    如果类在构造函数内分配资源
    则可能需要一个显示的复制构造函数来管理资源
    有析构函数（除了空虚析构函数之外）的类通常是用析构函数来书房构造函数分配的资源，这通常也说明需要一个复制构造函数
    eg：String类的复制构造函数
    class String{
        public:
            String();
            String(const char* s);
        private:
            char *data;
    };
    该类需要一个析构函数。因为它的数据成员指向了必须由对应的对象释放的被动态分配的内存。
    还需要一个显示的复制构造函数
    没有的话，复制String对象将会复制它的data成员的形式隐式地定义
    复制完后，两个对象的data成员将指向同样的内存，当这两个对象被销毁时，这个内存会被释放两次

    如果不想用户能够复制类的对象 需要声明复制构造函数为私有的
    private:
        Thing(const Ting&);
        Thing& operator=(const Thing&);
    如果其他成员不会使用这些函数，加上声明就ok 不用定义

    类的赋值操作符：
        如果不想用户能够设置类中的对象，就将赋值操作符私有化
        类的赋值由X::opertaor=来定义 通常 operator= 应该返回一个X& 并且由 return *this;结束 保证与内建的复制操作符一致

    赋值操作符正确地将对象赋给对象本身：
        赋值总是用新值取代目标对象的旧值
        如果原对象和目标对象是同一个。而实行“先释放旧值，再复制”就可能销毁原对象而没有复制
        eg:
            class String{
                public:
                    String& operator=(const String& s);
                private:
                    char* data;
            };
            String& String::operator=(const String& s){
                delete [] data;
                data = new char[strlen(s)+1];
                strcpy(data,s);
                return *this;
            }
        上面的方式： 将一个String对象赋给它本身时，此时s 和 *this指向的时同样的对象
        正确的方式： 在delete外面加一层判断 if(&s != this)
        另一个方法： 提前将旧的目标值保存起来 直到源值复制完成
        char* newData = new char[strlen(s) + 1];
        strcpy(newData,s.data);
        delete [] data;
        data = newData;
        return *this;

类定义关系操作符：
    提供operator== 或operator!=的重载以支持容器是否相等

delete [] 删除数组：
    cpp在释放数组时不一定要清楚数组的大小
    即使malloc把长度存储到某个位置上 cpp也不能保证找到这个值
    因此需要用户告知是不是删除的数组，该实现会提供另一个地方来存储长度
    因为与数组所需内存量相比，常数的开销会小很多

复制构造函数和赋值构造函数 的参数类型加const：
    X(const X& x)
    复制不会改变const
    绑定一个非const引用 到一个临时对象上是非法的

函数的引用参数 和const引用：
    当函数想改变参数 才应该有不用const声明的引用参数

适当声明成员函数为const：
    确信一个成员函数不会修改它的对象，就可以声明它为const
    这样就把它用于const对象了
    eg:
        public:
            int length() const;

    template<Class T>
    int padded_length(const vactor<T>& v,int n){
        int k = v.length();//编译出错
        return k > n ? k:n;
    }
    上面k的赋值就会发生错误，因为v是const引用
    需要在函数声明加上const

代理
允许将整个派生层次压缩在一个对象类型中

定于代理类
    使用类来表示概念
    定义一个行为和基类对象相似 又潜在地表示了所有派生类的对象的东西
    这种类的对象叫做代理 surrotage
    每个基类代理都代表某个派生类的对象，只要该代理关联着这个对象
    该对象就肯定存在
    因此，复制代理就会复制相对应的对象，而给代理赋新值也会删除旧对象，再复制新对象
    1.在基类中声明虚函数copy来完成复制工作
    2.定义代理类
    eg:
    class VehicleSurrogate{
        public:
            VehicleSurrogate();
            VehicleSurrogate(const Vehicle&);
            ~VehicleSurrogate();
            VehicleSurrogate(const VehicleSurrogate&);
            VehicleSurrogate& operator=(const VehicleSurrogate&);
        private:
            Vehicle* vp;
    };

    引入行为类似于零指针的空代理的概念
    能够创建 销毁和复制这样的代理
    但是进行其他操作就视为出错
    成员函数的定义：
        VehicleSurrogate::VehicleSurrogate():vp(0){}
        VehicleSurrogate::VehicleSurrogate(const Vehicle& v):vp(v.copy()){}
        VehicleSurrogate::~VehicleSurrogate(){
            delete vp;
        }
        VehicleSurrogate::VehicleSurrogate(const VehicleSurrogate& v):vp(v.vp? v.vp->copy():0){}
        VehicleSurrogate& VehicleSurrogate::operator=(const V& v){
            if(this!=v){
                delete vp;
                vp = (v.vp? v.vp->copy():0);
            }
            return *this;
        }
    每次对copy的调用都是一个虚拟调用，这些调用只能是虚拟的，因为类Vehicle的对象不存在
    v.copy是一次虚拟调用，因为v是一个引用而不是一个普通对象
    关于v.vp的非零检测。这个检测是必须的 否则调用v.vp->copy会出错
    注意对赋值操作符进行检测，确保没有将代理赋值给它自身
    VehicleSurrogate parking_lot[1000];
    Automobile x;
    Parking_lot[num_vehicles++] = x;
    //等价于
    Parking_lot[num_vehicles++] = VehicleSurrogate(x);
    创建了一个关于对象x的副本 并将VehicleSurrogate对象绑定到该副本
    然后将这个对象赋给parking_lot的一个元素 最后销毁parking_lot时 这些副本也将被清除

句柄
    在保持代理的多态行为的同时，可以避免进行不必要的复制
    因为有些对象会很大，复制起来消耗太大
    有可能某些其他的数据结构已经存储了对象的地址
    把副本的地址插入到那些数据结构中代价会很大

    另一方面，通常由于参数和返回值是通过复制自动传递的
    所以cpp函数可以很轻易地进行复制操作
    使用引用传递参数可以避免对它们的复制
    或者避免使用指针来复制对象。实际上 指针或者引用对于cpp的多态性是十分必要的

    未初始化的指针很危险，实施复制操作会导致彻底崩溃
    eg:
        int *p;//没有初始化
        int q=p;//未定义操作
        这样会导致硬件陷阱

    有几个指针指向同一个对象，必须考虑在什么时候删除这个对象，如果删除的太早，会有某个仍然指向它的指针存在。再使用这个指针会引发未定义行为
    如果删除对象太晚，会占用本来的内存
    可能会丢失指向该对象的指针 以至于无法释放这个对象

    cpp是定义一个适当的类 由于这些类的对象通常被绑定到它们所控制的类的对象上，这些类被成为handle类
    也叫智能指针
    但是handle某些情况下与智能指针视作相同

class Point{
    public:
        Point():xval(0),yval(0){}
        Point(int x,int y):xval(x),yval(y){}
        int x()const{ return xval;}
        int y()const{ return yval;}
        Point& x(int xv){
            xval = xv;
            return *this;
        }
        Point& y(int yv){
            yval = yv;
            return *this;
        }
    private:
        int xval,yval;
};

    一个无参的构造函数，用于在没有明确提供坐标的情况下创建一个Point

class Handle{
    public:
        Handle();
        ~Handle();
        Handle(int,int);
        Handle(const Point&);
        Handle(const Handle&);
        Handle& operator=(const Handle&);

        int x() const;
        int y() const;
        Handle& x(int);
        Handle& y(int);

};

引用计数型句柄
    使用句柄是为了避免不必要的对象复制
    即：允许多个句柄绑定到单个对象上
    通常需要了解有多少个句柄绑定在同一个对象上
    该引用计数不能是句柄的一部分，否则每个句柄都知道和它一个被绑定到同一个对象的其他所有句柄的位置

class Expr_node{
    friend ostream& operator<<(ostream&, const Expr_node&);

    protected:
        virtual void print(ostream&) const =0;
        virtual ~Expt_node(){}
};
创建的对象的类型派生自Expr_node
提供一个虚析构函数
保证在删除由一个Expr_node* 指针指向的对象时 能够调用到正确的派生类析构函数

需要动态绑定来处理print操作
使用<<输出操作符来打印表达式树
动态绑定只作用于成员函数，输出操作符可以调用它来完成实际的工作

同时希望用户使用输出操作符而不是print函数
将print函数设置为protected 将operator<<设置为友元

Expr_node是一个抽象基类
不存在Expr_node对象 只有派生类有对象
Expr_node的存在是为了获得公共接口

定义输出操作符：
    ostream&
    operator<<(ostream& o,const Expr_node& e){
        e.print(o);
        return o;
    }
没有子节点的节点类
class Int_node:public Expr_node{
    friend class Expr;

    int n;

    Int_node(int k):n(k){}
    void print(ostream& o) const { 
        o << n;
    }
};
按照指针存储子节点 通用类String代表操作符
一元和二元节点类：
class Unary_node:public Expr_node{
    friend class Expr;
    String op;
    Expr_node* opnd;
    Unary_node(const String& a,Expr_node* b):
        op(a),opnd(b){}
    void print(ostream& o) const{
        o<< "(" << op << *opnd << ")";
    }
};
class Binary_node:public Expr_node{
    friend class Expr;
    String op;
    Expr_node* left;
    Expr_node* right;
    Binary_node(const String&a,Expr_node*b , Expr_node* c):
        op(a),left(b),right(c){}
    
    void print(ostream& o)const{
        o<< "(" << *left << op << *right << ")";
    }
};
上面用户需要处理指针
所以需要分配和释放对象
创建表达式树：
    Expr t = Expr("*",Expr("-",5),Expr("+",3,4));
    这种创建方式错误 一元和二元表达式的构造函数期望获得指针 而不是对象
    动态分配节点：
        Binary_node * t = new Binary_node("*",
                            new Unary_node("-",new Int_node(5)),
                            new Binary_node("+",new Int_node(3),new Int_node(4)));
        但是这种创建方式会有释放对象问题  造成内存无法释放  

Expr构造函数能代表3种Expr_node        
每个Expr构造函数都将创建Expr_node派生类的一个合适对象，并且将这个对象的地址存储在正在创建中的Expr对象中
Expr类的用户不会直接看到Expr_node对象:
    class Expr{
        friend ostream& operator<<(ostream&,const Expr&);

        Expr_node* p;
        public:
            Expr(int);
            Expr(const String&,Expr);
            Expr(const String&,Expr,Expr);
            Expr(const Expr&);
            Expr& operator=(const Expr&);
            ~Expr(){delete p;}
    };       
Expr::Expr(int n){
    p = new Int_node(n);
} 
Expr::Expr(const String& op,Expr t){
    p = new Unary_node(op,t);
}
Expr::Expr(const String& op,Expr left,Expr right){
    p = new Binary_node(op,left,right);
}
析构函数是放在构造函数中分配的节点
Expr构造函数为Expr_node分配了内存，需要实现复制构造函数和赋值操作符管理下层的Expr_node
如果Expr的析构函数销毁了p所指的对象
在复制或赋值一个Expr时 需要生成该对象的一个副本
可以想Expr_node派生层次中加入 一个虚函数copy 在Expr对象中使用它

Expr的操作不会改变下层的Expr_node
避免复制下层的Expr_node会更有效率

避免复制的常用办法是让每一个Expr_node包含一个引用计数
指明同时由多少Expr指向同一个Expr_node
Expr_node类和Expr类共同管理引用计数
当且仅当一个expr_node的引用计数等于0时，该节点才被删除

需要在Expr_node中加入引用计数
当一个新的expr_node派生类对象生成时，将引用计数初始化为1
Expr类帮助管理引用计数
在Expr_node中声明为友元
    class Expr_node{
        friend ostream& operator<<(ostream &,const Expr&);
        friend class Expr;

        int use;
    
        protected:
            Expr_node():use(1){}
            virtual void print(ostream&) const=0;
            virtual ~Expr_node(){}
    };
    Expr类“复制”一个Expr_node 该Expr将其引用计数增1
    当引用计数为0时删除底层的Expr_node
    class Expr{
        //新增部分  
        public:
            Expr(const Expr& t){
                p = t.p;
                ++p->use;
            }
            ~Expr(){
                if(--p->use == 0){
                    delete p;
                }
            }
            Expr& operator=(const Expr& t);
    };
    复制构造函数递增引用计数
    令p指向其复制目标指向的同一个Expr_node
    析构减少引用计数
    
    赋值操作符比附分别递增和递减右边和左边对象的引用计数
    首先处理右边对象的引用计数，保证在自我赋值的情况下工作正常
    Expr&
    Expr::operator=(const Expr&  rhs){
        rhs.p->use++;
        if(--p->use == 0){
            delete p;
        }
        p = rhs.p;
        return *this;
    }
    定义输出操作符 针对Expr操作 体现了额外的中间层
    ostream&
    operator<<(ostream &o,const Expr& t){
        t.p->print(o);
        return o;
    }
    最后 更改每个派生自Expr_node的类
    令其操作为私有 将Expr类声明为友元
    存储Expr而不是存储指向Expr_node的指针
    例如之前的Binary_node {
        friend class Expr;
        Expr left;
        Expr right;
    }
    此时 无需考虑内存管理的问题，用户可以构造任意复杂的表达式

计算Expr表达式的结果
Expr增加:
    int eval()const { return p->eval();}
Expr_node 增加一个虚函数:
    vitrual int eval()const =0;
Int_node和Unary_node Binary_node增加：
    int eval()const{
        return n;
    }
    int eval()const{
        if(op == "-"){
            return -opnd.eval();
        } 
        throw "erro,bad op" + op + "int UnaryNode";
    }
    int eval()const{
        int op1 = left.eval();
        int op2 = right.eval();
        if("+" == op) return op1+op2;
        if("-" == op) return op1-op2;
        if("*" == op) return op1*op2;
        if("/" == op && op2!=0) return op1/op2;

        throw "error, bad op " + op + "in BinaryNode";

    }

增加新的节点类型
比如三目运算符?:
增加一个类
class Trinary_node:public class Expr_node{
    friend class Exxpr;

    String op;
    Expr left;
    Expr middle;
    Expr right;

    Trinary_node(const String& a,Expr b, Expr c,Expr d):
        op(a),left(a),middle(b),right(c){}
    void print(ostream& o) const{
        o << "(" << left << "?" << middle << ":" << right << ")";
    }
    int eval()const{
        if(left.eval()){
            return middle.eval();
        }
        return right.eval();
    }
};
Expr_node添加：
    Expr_node(const String &,Expr,Expr,Expr);
Expr添加：
    Expr(cosnt String& op,Expr a,Expr b,Expr c){
        p = new Trinary_node(op,a,b,c);
    }


一个类需要一个指针 指向包含字符本身的辅助内存
意味着需要一个复制构造函数 析构函数 和赋值操作符
为了能定义类对象数组
需要一个缺省构造函数

cpp处理二维数组 行*列 的一维数组
class Picture{
    private:
        int height,width;
        char* data;
        //允许直接读写给定字符的左值
        char& position(int row,int col){
            return data[row*width + col];
        }
        //返回所需字符的副本
        char position(int row,int col) const{
            return data[row*width + col];
        }
};

类支持继承 需要虚析构函数
虚函数和非虚函数之间的区别在特定环境才会体现出来：
    当使用一个基类指针或引用 来指向或引用 一个派生类对象时
    class Base{
        public:
            void f();
            virtual void g();
    };
    class Dirived:public Base{
        public:
            void f();
            virtual void g();
    };
    创建Base类的对象和Derived类的对象 获得指向它们的指针
    Base b;
    Derived d;
    Base* bp = &b; 指向Base对象
    Base* bq = &d;
    Derived* dp = &d;
    调用成员函数 f 和 g
    bp->f();Base::f
    bq->f();Base::f
    dp->f();Derived::f

    bp->g();Base::g
    bq->g();Derived::g
    dp->g();Derived::g

    指针的静态类型与它所指向的实际对象不同时
    非虚函数和虚函数f g 运行起来有差别
    
    需要虚析构函数的情况：
        有需要析构发生
        发生在这样的上下文中：指向一个基类的指针或引用都有一个静态类型，并且实际上都指向一个派生类的对象

    基类中有虚析构函数 派生类自动继承 不用声明第二个
    删除一个表面指向基类对象 实际却指向派生类对象的指针 就需要虚析构函数

不考虑派生问题、虚函数的消耗 --虚函数不总是适用

容器包含对象
    假设一个对象放到两个容器c1 c2中
    对象中有一个成员函数可以改变它的值
    先将对象insert到c1容器
    再调用对象的成员函数
    是否会改变容器中对象的值 or 插入的是对象的副本

    另一个：
    向c1 c2中insert同一个对象
    改变c1中该对象的值 c2中该对象的值是否改变

    相关问题 对象释放：
    void f(){
        container c{
            Object ob;
            c.insert(ob);
        }
        //c是否还有效
    }

    为什么不把指向对象的指针 而不是对象本身放入容器中

    总结：容器应该包含放在其中的对象的副本 而不是原对象本身

    想在多个容器中包含一个对象的用户——即保存对象标识：
        可以将指向该对象的指针放入到容器中

通常将容器称为模板 容器类的对象类型就是 模板参数
对于任意类型T 可以想象容器可以是 List<T> 或者Set<T>
不关心容器类型即 Container<T>
如果向容器中放入对象需要复制对象 复制容器是否该复制包含在容器中的对象
    

结构体的复制或赋值：值被复制
对于C数组： 不允许数组间的复制或赋值 ，只有在函数调用时传入一个数组 实际是传的指针 会改变原数组中的值
总结：
    结构体实现值语义： 复制完成后，两个变量都有这个值的独立的副本
    数组实现引用传递：复制完成后，两个变量都引用同一个底层对象

传递容器 加上const和& 避免改变容器中对象的值
因为复制容器就意味着复制容器中每个元素

指针和下标的不同：
    放在容器中说
    下标值本身有意义，与它是否用作下标有关
    访问容器元素没必要知道容器标识，指针自身包含了相关问题
普通的不同：指针效率更高 下标易于理解

类里写多线程
std::thread thrd = std::thread(&class::function,this)
析构里 join()